In [1]:
sc

Intitializing Scala interpreter ...

Spark Web UI available at http://172.16.8.92:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1583741685962)
SparkSession available as 'spark'


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@2321e284


In [2]:
val path = "/Users/soda/Downloads/backUp/Notebooks/AdvancedAnalyticswithSpark/data/"
val dataWithoutHeader = spark.read.
    option("inferSchema",true).
    option("header",false).
    csv(path+"covtype.data")

path: String = /Users/soda/Downloads/backUp/Notebooks/AdvancedAnalyticswithSpark/data/
dataWithoutHeader: org.apache.spark.sql.DataFrame = [_c0: int, _c1: int ... 53 more fields]


In [3]:
val colNames = Seq(
    "Elevation", "Aspect", "Slope",
    "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
  ) ++ ( 
    (0 until 4).map(i => s"Wilderness_Area_$i")
  ) ++ (
    (0 until 40).map(i => s"Soil_Type_$i")
  ) ++ Seq("Cover_Type")

colNames: Seq[String] = List(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil_...

In [4]:
// 在 Spark MLlib 所有 API 中，目标列通常都被视为双精度浮点数类型而不是整型
val data = dataWithoutHeader.toDF(colNames:_*).
    withColumn("Cover_Type",$"Cover_Type".cast("double"))

data: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 53 more fields]


In [5]:
data.head

res1: org.apache.spark.sql.Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5.0]


4.7　第一棵决策树

In [6]:
val Array(trainData,testData) = data.randomSplit(Array(0.9,0.1))
trainData.cache()
testData.cache()

trainData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 53 more fields]
res2: testData.type = [Elevation: int, Aspect: int ... 53 more fields]


Spark MLlib 要求将所有输入合并成一列，该列的值是一个向量。这个类是线性代数中向量的一个抽象，仅包含一些数字。对于大多数的意图和目的，向量就像是一个简单的双精度浮点数数组。当然，有些输入特征在概念上是类别型的，即使在输入中它们都是用数值表示的。

In [7]:
import org.apache.spark.ml.feature.VectorAssembler

val inputCols = trainData.columns.filter( _!="Cover_Type")
val assembler = new VectorAssembler().
    setInputCols(inputCols).
    setOutputCol("featureVector")

val assemblerTrainData = assembler.transform(trainData)

import org.apache.spark.ml.feature.VectorAssembler
inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_T...

In [8]:
assemblerTrainData.select("featureVector").show(truncate=false)

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175.0,195.0,224.0,168.0,732.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1898.0,34.0,23.0,175.0,56.0,134.0,210.0,184.0,99.0,765.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1899.0,355.0,22.0,153.0,43.0,1

In [11]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random

val classifier = new DecisionTreeClassifier().
    setSeed(Random.nextLong()).
    setLabelCol("Cover_Type").
    setFeaturesCol("featureVector").
    setPredictionCol("prediction")

val model = classifier.fit(assemblerTrainData)

import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random
classifier: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_70fcec132ebe
model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_70fcec132ebe) of depth 5 with 47 nodes


In [12]:
println(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_70fcec132ebe) of depth 5 with 47 nodes
  If (feature 0 <= 3047.5)
   If (feature 0 <= 2514.5)
    If (feature 10 <= 0.5)
     If (feature 3 <= 15.0)
      If (feature 12 <= 0.5)
       Predict: 4.0
      Else (feature 12 > 0.5)
       Predict: 6.0
     Else (feature 3 > 15.0)
      Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 9 <= 5482.5)
      Predict: 2.0
     Else (feature 9 > 5482.5)
      If (feature 5 <= 621.0)
       Predict: 2.0
      Else (feature 5 > 621.0)
       Predict: 5.0
   Else (feature 0 > 2514.5)
    If (feature 17 <= 0.5)
     If (feature 0 <= 2951.5)
      If (feature 15 <= 0.5)
       Predict: 2.0
      Else (feature 15 > 0.5)
       Predict: 3.0
     Else (feature 0 > 2951.5)
      Predict: 2.0
    Else (feature 17 > 0.5)
     If (feature 0 <= 2708.5)
      Predict: 3.0
     Else (feature 0 > 2708.5)
      If (feature 5 <= 1178.0)
       Predict: 5.0
      Else (feature 5 > 1178.0)
       Predict: 2.0
  Else (

In [13]:
model.featureImportances.toArray.zip(inputCols).
    sorted.reverse.foreach(println)

(0.8340238829800503,Elevation)
(0.032560153307420445,Soil_Type_3)
(0.02992060724618838,Soil_Type_1)
(0.02597536404540168,Hillshade_Noon)
(0.02497609703087602,Soil_Type_31)
(0.023110508973576863,Horizontal_Distance_To_Hydrology)
(0.015196520961657254,Wilderness_Area_2)
(0.00669560030172463,Wilderness_Area_0)
(0.004565310672848323,Horizontal_Distance_To_Roadways)
(0.0026181830974530335,Hillshade_9am)
(3.577713828031636E-4,Horizontal_Distance_To_Fire_Points)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_9)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soil_Type_22)
(0.0,Soil_Type_21)
(0.0,

In [14]:
val predictions = model.transform(assemblerTrainData)

predictions: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 57 more fields]


In [16]:
predictions.select("Cover_Type","prediction","probability").show(5,truncate=false)

+----------+----------+-----------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                      |
+----------+----------+-----------------------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,2.794076557697681E-5,0.07066219614417435,0.6050013970382788,0.040765576976809166,0.0,0.2835428890751607,0.0]|
|6.0       |4.0       |[0.0,0.0,0.06932668329177058,0.2937655860349127,0.5032418952618454,0.0,0.1336658354114713,0.0]                   |
|6.0       |3.0       |[0.0,2.794076557697681E-5,0.07066219614417435,0.6050013970382788,0.040765576976809166,0.0,0.2835428890751607,0.0]|
|6.0       |3.0       |[0.0,2.794076557697681E-5,0.07066219614417435,0.6050013970382788,0.040765576976809166,0.0,0.2835428890751607,0.0]|
|6.0       |3.0       |[0.0,2.7940

In [17]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val evaluator = new MulticlassClassificationEvaluator().
    setLabelCol("Cover_Type").
    setPredictionCol("prediction")

evaluator.setMetricName("accuracy").evaluate(predictions)
evaluator.setMetricName("f1").evaluate(predictions)

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9ceaef7f005f
res8: Double = 0.6887950370053781


In [18]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val predictionRDD = predictions.
    select("prediction","Cover_Type").
    as[(Double,Double)].
    rdd

val multiclassMetrics = new MulticlassMetrics(predictionRDD)


import org.apache.spark.mllib.evaluation.MulticlassMetrics
predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[103] at rdd at <console>:35
multiclassMetrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@565847fd


In [19]:
multiclassMetrics.confusionMatrix

res9: org.apache.spark.mllib.linalg.Matrix =
126481.0  58304.0   98.0     0.0     24.0   8.0    5670.0
48460.0   201271.0  3831.0   139.0   298.0  65.0   845.0
0.0       3581.0    27926.0  589.0   25.0   149.0  0.0
0.0       3.0       1459.0   1009.0  0.0    0.0    0.0
0.0       7834.0    282.0    0.0     430.0  0.0    0.0
0.0       4058.0    10789.0  268.0   8.0    587.0  0.0
7855.0    76.0      0.0      0.0     0.0    0.0    10570.0


In [21]:
val confusionMatrix = predictions.groupBy("Cover_Type").
    pivot("prediction",(1 to 7)).
    count().
    na.fill(0.0).
    orderBy("Cover_Type")

confusionMatrix: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Cover_Type: double, 1: bigint ... 6 more fields]


In [22]:
confusionMatrix.show()

+----------+------+------+-----+----+---+---+-----+
|Cover_Type|     1|     2|    3|   4|  5|  6|    7|
+----------+------+------+-----+----+---+---+-----+
|       1.0|126481| 58304|   98|   0| 24|  8| 5670|
|       2.0| 48460|201271| 3831| 139|298| 65|  845|
|       3.0|     0|  3581|27926| 589| 25|149|    0|
|       4.0|     0|     3| 1459|1009|  0|  0|    0|
|       5.0|     0|  7834|  282|   0|430|  0|    0|
|       6.0|     0|  4058|10789| 268|  8|587|    0|
|       7.0|  7855|    76|    0|   0|  0|  0|10570|
+----------+------+------+-----+----+---+---+-----+



In [25]:
import org.apache.spark.sql.DataFrame

def classProbabilities(data: DataFrame): Array[Double] = {
    val total = data.count()
    data.groupBy("Cover_Type").count().
        orderBy("Cover_Type").
        select("count").as[Double].
        map(_ / total).
        collect()
}

import org.apache.spark.sql.DataFrame
classProbabilities: (data: org.apache.spark.sql.DataFrame)Array[Double]


In [26]:
val trainPriorProb = classProbabilities(trainData)
val testPriorProb = classProbabilities(testData)

trainPriorProb: Array[Double] = Array(0.36441283996695933, 0.4874051610732095, 0.06170266466791079, 0.004724737663291217, 0.016340594119986538, 0.03003870040077095, 0.03537530210787163)
testPriorProb: Array[Double] = Array(0.36633919338159254, 0.4893485005170631, 0.06004825922095829, 0.004756980351602895, 0.016321957945536023, 0.028559117545673904, 0.03462599103757325)
